In [1]:
import os
import pandas as pd
import numpy as np

# Data directory
#data_path = "data/CREMP_CSV_files"
data_path = "/Users/olaoluwatunmise/Coral-Quest-Florida-Keys/data/CREMP_CSV_files"
# Get all CSV filenames
csv_files = [file for file in os.listdir(data_path)
             if file.endswith('.csv')]

# Load all CSV files into a dictionary of DataFrames
data = {}

for file in csv_files:
    name = file.replace('.csv', '') #remove .csv extension for key name
    df = pd.read_csv(os.path.join(data_path, file))
    data[name] = df

In [2]:
csv_files

['CREMP_OCTO_RawData_2023.csv',
 'CREMP_SCOR_Summaries_2023_Counts.csv',
 'CREMP_SCOR_Summaries_2023_ConditionCounts.csv',
 'CREMP_OCTO_Summaries_2023_MeanHeight.csv',
 'CREMP_OCTO_Summaries_2023_Density.csv',
 'CREMP_SCOR_Summaries_2023_LTA.csv',
 'CREMP_Pcover_2023_StonyCoralSpecies.csv',
 'CREMP_Stations_2023.csv',
 'CREMP_SCOR_Summaries_2023_Density.csv',
 'CREMP_SCOR_RawData_2023.csv',
 'CREMP_Temperatures_2023.csv',
 'CREMP_Pcover_2023_TaxaGroups.csv']

In [3]:
# List of specific dataframes to inspect
specific_dfs = [
    'CREMP_OCTO_RawData_2023',
    'CREMP_OCTO_Summaries_2023_MeanHeight',
    'CREMP_OCTO_Summaries_2023_Density',
    'CREMP_SCOR_RawData_2023',
    'CREMP_SCOR_Summaries_2023_Counts',
    'CREMP_SCOR_Summaries_2023_ConditionCounts',
    'CREMP_SCOR_Summaries_2023_LTA',
    'CREMP_SCOR_Summaries_2023_Density',
    'CREMP_Pcover_2023_TaxaGroups',
    'CREMP_Pcover_2023_StonyCoralSpecies'
]

# Revised inspection function
def inspect_df(name, df):
    print(f"\nDataset: {name}")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Look for columns resembling 'StationID' and 'Date'
    station_col = next((col for col in df.columns if 'station' in col.lower() or 'site' in col.lower()), None)
    date_col = next((col for col in df.columns if 'date' in col.lower() or 'year' in col.lower()), None)
    
    if station_col and date_col:
        unique_combos = df[[station_col, date_col]].drop_duplicates().shape[0]
        print(f"Unique {station_col}-{date_col} combos: {unique_combos}")
    else:
        print("Warning: Could not find StationID and/or Date-like columns!")
    
    print(f"Sample:\n{df.head(2)}")

# Inspect only the specified dataframes
for name in specific_dfs:
    if name in data:
        inspect_df(name, data[name])
    else:
        print(f"\nDataset {name} not found in 'data' dictionary!")


Dataset: CREMP_OCTO_RawData_2023
Rows: 109246, Columns: 10
Columns: ['Year', 'Date', 'Subregion', 'Habitat', 'SiteID', 'Site_name', 'StationID', 'SPP_Code', 'sciName', 'Height_cm']
Unique SiteID-Year combos: 257
Sample:
   Year      Date Subregion Habitat  SiteID     Site_name  StationID SPP_Code  \
0  2011  6/8/2011        UK       P      32  Porter Patch        321     PAME   
1  2011  6/8/2011        UK       P      32  Porter Patch        321     PAME   

                       sciName  Height_cm  
0  Pseudopterogorgia americana        9.0  
1  Pseudopterogorgia americana       54.0  

Dataset: CREMP_OCTO_Summaries_2023_MeanHeight
Rows: 1023, Columns: 13
Columns: ['Year', 'FirstOfDate', 'subRegionID', 'HabitatID', 'siteid', 'site_name', 'stationid', 'Eunicea_calyculata', 'Gorgonia_ventalina', 'Pseudopterogorgia_americana', 'Pseudopterogorgia_bipinnata', 'Eunicea_flexuosa', 'Pseudoplexaura_porosa']
Unique siteid-Year combos: 257
Sample:
   Year FirstOfDate subRegionID HabitatID  si

# Merge Strategy 

## Core Coral Metrics

#### Stony Corals(SCOR) + Octocorals (OCTO)

In [4]:
data['CREMP_OCTO_Summaries_2023_MeanHeight'] = data['CREMP_OCTO_Summaries_2023_MeanHeight'].rename(
    columns={'stationid': 'StationID'}
)

data['CREMP_OCTO_Summaries_2023_MeanHeight'] = data['CREMP_OCTO_Summaries_2023_MeanHeight'].rename(
    columns={'FirstOfDate': 'Date'}
)


In [5]:
# Define columns to keep for each summary DataFrame
summary_columns = {
    'CREMP_OCTO_RawData_2023': ['StationID', 'Date', 'Habitat', 'Subregion', 'SPP_Code', 'SiteID', 'Height_cm'],
    'CREMP_SCOR_RawData_2023': ['StationID', 'Date', 'Diameter_cm','Height_cm', 'Percent_old_mortality', 
                                'Percent_recent_mortality','LTA_cm2' ],
    'CREMP_OCTO_Summaries_2023_MeanHeight': ['StationID', 'Date', 'Pseudopterogorgia_americana'],
    'CREMP_OCTO_Summaries_2023_Density': ['StationID', 'Date', 'Total_Octocorals'],
    'CREMP_SCOR_Summaries_2023_Counts': ['StationID', 'Date', 'Siderastrea_siderea'],
    'CREMP_SCOR_Summaries_2023_ConditionCounts': ['StationID', 'Date', 'sciName', 'Count'],
    'CREMP_SCOR_Summaries_2023_LTA': ['StationID', 'Date', 'Siderastrea_siderea'],
    'CREMP_SCOR_Summaries_2023_Density': ['StationID', 'Date', 'Siderastrea_siderea'],
    'CREMP_Pcover_2023_TaxaGroups': ['StationID', 'Date', 'SiteID', 'Site_name', 'Octocoral','Stony_coral', 'Substrate'],
    'CREMP_Pcover_2023_StonyCoralSpecies':['StationID', 'Date', 'Siderastrea_siderea']
}

# Trim columns
for df_name, cols in summary_columns.items():
    data[df_name] = data[df_name][cols]


# Optimize Data Types

In [6]:
for key in data.keys():
    # Convert StationID to categorical
    if 'StationID' in data[key].columns:
        data[key]['StationID'] = data[key]['StationID'].astype('category')
    
    # Convert Date to datetime
    if 'Date' in data[key].columns:
        data[key]['Date'] = pd.to_datetime(data[key]['Date'])

/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_43938/63792658.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[key]['StationID'] = data[key]['StationID'].astype('category')
/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_43938/63792658.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[key]['Date'] = pd.to_datetime(data[key]['Date'])


# Temporal Trends

### Coral Health Indicators

All summary data and PCover Data 

In [7]:
coral_health = data['CREMP_OCTO_Summaries_2023_MeanHeight']
    

# Merge OCTO summaries


coral_health = coral_health.merge(
    data['CREMP_OCTO_Summaries_2023_Density'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_octo_density')
)

# Merge SCOR summaries
coral_health = coral_health.merge(
    data['CREMP_SCOR_Summaries_2023_Counts'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_scor_counts')
)

coral_health = coral_health.merge(
    data['CREMP_SCOR_Summaries_2023_LTA'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_scor_lta')
)

coral_health = coral_health.merge(
    data['CREMP_SCOR_Summaries_2023_Density'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_scor_density')
)

coral_health = coral_health.merge(
    data['CREMP_SCOR_Summaries_2023_ConditionCounts'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_scor_condition')
)

# Merge PCover
coral_health = coral_health.merge(
    data['CREMP_Pcover_2023_TaxaGroups'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_taxa_cover')
)

core_coral = coral_health.merge(
    data['CREMP_Pcover_2023_StonyCoralSpecies'],
    on=['StationID', 'Date'],
    how='left',
    suffixes=('', '_scor_cover')
)

core_coral.head()

,StationID,Date,Pseudopterogorgia_americana,Total_Octocorals,Siderastrea_siderea,Siderastrea_siderea_scor_lta,Siderastrea_siderea_scor_density,sciName,Count,SiteID,Site_name,Octocoral,Stony_coral,Substrate,Siderastrea_siderea_scor_cover
0,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,Porter Patch,0.1322,0.0416,0.4957,0.0288
1,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Montastraea cavernosa,1.0,32.0,Porter Patch,0.1322,0.0416,0.4957,0.0288
2,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Porites astreoides,4.0,32.0,Porter Patch,0.1322,0.0416,0.4957,0.0288
3,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Porites porites,8.0,32.0,Porter Patch,0.1322,0.0416,0.4957,0.0288
4,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Siderastrea radians,1.0,32.0,Porter Patch,0.1322,0.0416,0.4957,0.0288


In [8]:
core_coral = core_coral.ffill().infer_objects(copy=False)


/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_43938/2787327091.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  core_coral = core_coral.ffill().infer_objects(copy=False)


In [9]:
core_coral.isna().sum()

%store core_coral

Stored 'core_coral' (DataFrame)


# Spatial Pattern Analysis

Stony Corel Raw data + Octo Coral Raw data

# Merging Octo RawData with SCOR RawData

In [10]:
octo = data['CREMP_OCTO_RawData_2023']

octo_sto = octo.merge(data['CREMP_SCOR_RawData_2023'],
                       on=['StationID', 'Date'],
                       how='left',
                       suffixes=('', '_scor'))


octo_sto.head()



,StationID,Date,Habitat,Subregion,SPP_Code,SiteID,Height_cm,Diameter_cm,Height_cm_scor,Percent_old_mortality,Percent_recent_mortality,LTA_cm2
0,321,2011-06-08,P,UK,PAME,32,9.0,4.0,1.0,0.0,0.0,17.36
1,321,2011-06-08,P,UK,PAME,32,9.0,7.0,2.0,0.0,0.0,56.27
2,321,2011-06-08,P,UK,PAME,32,9.0,9.0,3.0,0.0,0.0,100.16
3,321,2011-06-08,P,UK,PAME,32,9.0,12.0,8.0,15.0,0.0,236.18
4,321,2011-06-08,P,UK,PAME,32,9.0,5.0,1.0,0.0,0.0,25.07


In [11]:
octo_sto = octo_sto.ffill().infer_objects(copy=False)

In [12]:
octo_sto.head()

%store octo_sto

Stored 'octo_sto' (DataFrame)


## Get Site Coordinates

Merged Octo and Stony Coral Raw Data with Station Data for mapping

In [13]:
# Get site coordinates + habitat + depth
stations_df = data['CREMP_Stations_2023'].copy()

# Ensure consistent types for merging
stations_df['SiteID'] = stations_df['SiteID'].astype(str)
octo_sto['SiteID'] = octo_sto['SiteID'].astype(str)

# Merge relevant station info
octo_sto_coords = octo_sto.merge(
    stations_df[['SiteID', 'Site_name', 'latDD', 'lonDD', 'Depth_ft', 'length_m', 'Habitat']],
    on='SiteID',
    how='left'
)


octo_sto_coords.tail()

,StationID,Date,Habitat_x,Subregion,SPP_Code,SiteID,Height_cm,Diameter_cm,Height_cm_scor,Percent_old_mortality,Percent_recent_mortality,LTA_cm2,Site_name,latDD,lonDD,Depth_ft,length_m,Habitat_y
21674375,814,2023-06-08,OD,LK,PFLE,81,8.0,9.0,1.0,0.0,0.0,71.13,Sand Key Deep,24.4517,-81.8798,30,22,OD
21674376,814,2023-06-08,OD,LK,PFLE,81,8.0,16.0,4.0,5.0,0.0,263.99,Sand Key Deep,24.4517,-81.8798,35,22,OD
21674377,814,2023-06-08,OD,LK,PFLE,81,8.0,16.0,4.0,5.0,0.0,263.99,Sand Key Deep,24.4517,-81.8798,34,22,OD
21674378,814,2023-06-08,OD,LK,PFLE,81,8.0,16.0,4.0,5.0,0.0,263.99,Sand Key Deep,24.4517,-81.8798,33,22,OD
21674379,814,2023-06-08,OD,LK,PFLE,81,8.0,16.0,4.0,5.0,0.0,263.99,Sand Key Deep,24.4517,-81.8798,30,22,OD


## Group Sites by Depth 
 Categorize by shallow vs. deep :

In [14]:
octo_sto_coords['Reef_Type'] = octo_sto_coords['Depth_ft'].apply(
    lambda x: 'Shallow' if x < 20 else 'Deep'
)


In [15]:
# Forward fill missing values
octo_sto_coords = octo_sto_coords.ffill()

In [16]:
octo_sto_coords.isna().sum()

%store octo_sto_coords

Stored 'octo_sto_coords' (DataFrame)


In [17]:
# import plotly.express as px

# fig = px.scatter_geo(
#     octo_sto_coords.dropna(subset=['latDD', 'lonDD']),
#     lat='latDD',
#     lon='lonDD',
#     color='Habitat_y',
#     hover_name='Site_name',
#     size='LTA_cm2',  # Optional: can use any coral metric
#     projection='natural earth',
#     title='Coral Metrics Across CREMP Sites'
# )
# fig.show()


# Environmental Correlations 




In [18]:


temp_df = data['CREMP_Temperatures_2023']



# Combine columns into a datetime
temp_df['Date'] = pd.to_datetime(
    temp_df[['Year', 'Month', 'Day']].astype(str).agg('-'.join, axis=1)  #+ ' ' + temp_df['Time'].astype(str)
)

temp_df.head()

# # env_spatial = env_spatial.merge(data['CREMP_Stations_2023'], on=['OID_'], how='left')

# # env_spatial.head()


,OID_,SiteID,Site_name,Year,Month,Day,Time,TempC,TempF,Date
0,1,10,Rattlesnake,2020,6,12,11.0,29.59,85.26,2020-06-12
1,2,10,Rattlesnake,2020,6,12,12.0,29.76,85.57,2020-06-12
2,3,10,Rattlesnake,2020,6,12,13.0,29.81,85.66,2020-06-12
3,4,10,Rattlesnake,2020,6,12,14.0,30.19,86.34,2020-06-12
4,5,10,Rattlesnake,2020,6,12,15.0,30.34,86.61,2020-06-12


In [19]:
core_coral_env = core_coral.merge(temp_df,
                       on=['SiteID', 'Date'],
                       how='left',
                       suffixes=('', '_scor'))

core_coral_env.head()

,StationID,Date,Pseudopterogorgia_americana,Total_Octocorals,Siderastrea_siderea,Siderastrea_siderea_scor_lta,Siderastrea_siderea_scor_density,sciName,Count,SiteID,...,Substrate,Siderastrea_siderea_scor_cover,OID_,Site_name_scor,Year,Month,Day,Time,TempC,TempF
0,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,...,0.4957,0.0288,1378434.0,Porter Patch,2011.0,6.0,8.0,0.0,27.80,82.04
1,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,...,0.4957,0.0288,1378435.0,Porter Patch,2011.0,6.0,8.0,1.0,27.65,81.77
2,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,...,0.4957,0.0288,1378436.0,Porter Patch,2011.0,6.0,8.0,2.0,27.55,81.59
3,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,...,0.4957,0.0288,1378437.0,Porter Patch,2011.0,6.0,8.0,3.0,27.40,81.33
4,321,2011-06-08,23.8,9.0,18.0,7324.5,1.8,Dichocoenia stokesii,4.0,32.0,...,0.4957,0.0288,1378438.0,Porter Patch,2011.0,6.0,8.0,4.0,27.43,81.37


In [20]:
core_coral_env.isna().sum()

StationID                             0
Date                                  0
Pseudopterogorgia_americana           0
Total_Octocorals                      0
Siderastrea_siderea                   0
Siderastrea_siderea_scor_lta          0
Siderastrea_siderea_scor_density      0
sciName                               0
Count                                 0
SiteID                                0
Site_name                             0
Octocoral                             0
Stony_coral                           0
Substrate                             0
Siderastrea_siderea_scor_cover        0
OID_                                627
Site_name_scor                      627
Year                                627
Month                               627
Day                                 627
Time                                627
TempC                               627
TempF                               675
dtype: int64

In [21]:
# Drop unwanted columns
core_coral_env = core_coral_env.drop(columns=['Site_name_scor', 'OID_', 'Year', 'Month', 'Day', 'Time'])

# Forward fill temperature columns only
core_coral_env[['TempC', 'TempF']] = core_coral_env[['TempC', 'TempF']].ffill().infer_objects(copy=False)

core_coral_env.isna().sum()

StationID                           0
Date                                0
Pseudopterogorgia_americana         0
Total_Octocorals                    0
Siderastrea_siderea                 0
Siderastrea_siderea_scor_lta        0
Siderastrea_siderea_scor_density    0
sciName                             0
Count                               0
SiteID                              0
Site_name                           0
Octocoral                           0
Stony_coral                         0
Substrate                           0
Siderastrea_siderea_scor_cover      0
TempC                               0
TempF                               0
dtype: int64

In [22]:
core_coral_env.head()

%store core_coral_env

Stored 'core_coral_env' (DataFrame)


### DataFrames for EDA

In [23]:
# # Temporal Trends
# core_coral
# # Spatial Patterns
# octo_sto
# # Site Coordination
# octo_sto_coords
# # Environmental Correlations
# core_coral_env


In [24]:
# import joblib
# import os

# # Define your folder path
# folder_path = "/Users/olaoluwatunmise/Coral-Quest-Florida-Keys/data/merged-data"

# # Ensure the folder exists
# os.makedirs(folder_path, exist_ok=True)

# # Dictionary of DataFrames
# dataframes = {
#     "core_coral": core_coral,
#     "octo_sto": octo_sto,
#     "octo_sto_coords": octo_sto_coords,
#     "core_coral_env": core_coral_env
# }

# # Save each DataFrame
# for name, df in dataframes.items():
#     joblib.dump(df, os.path.join(folder_path, f"{name}.joblib"))


#### Use Stations as a Lookup Table

- Keep station metadata separate and map it to coral data only when needed

- Grouping & Aggregation: Count coral colonies by subregion or habitat type